In [30]:
!pip3 install beautifulsoup4
!pip3 install requests

In [31]:
import sys

import requests
from bs4 import BeautifulSoup
import re
import unicodedata
import pandas as pd

In [32]:
def date_time(table_cells):
    """
    This function returns the data and time from the HTML  table cell
    Input: the  element of a table data cell extracts extra row
    """
    return [data_time.strip() for data_time in list(table_cells.strings)][0:2]

def booster_version(table_cells):
    """
    This function returns the booster version from the HTML  table cell 
    Input: the  element of a table data cell extracts extra row
    """
    out=''.join([booster_version for i,booster_version in enumerate( table_cells.strings) if i%2==0][0:-1])
    return out

def landing_status(table_cells):
    """
    This function returns the landing status from the HTML table cell 
    Input: the  element of a table data cell extracts extra row
    """
    out=[i for i in table_cells.strings][0]
    return out


def get_mass(table_cells):
    mass=unicodedata.normalize("NFKD", table_cells.text).strip()
    if mass:
        mass.find("kg")
        new_mass=mass[0:mass.find("kg")+2]
    else:
        new_mass=0
    return new_mass


def extract_column_from_header(row):
    """
    This function returns the landing status from the HTML table cell 
    Input: the  element of a table data cell extracts extra row
    """
    if (row.br):
        row.br.extract()
    if row.a:
        row.a.extract()
    if row.sup:
        row.sup.extract()
        
    colunm_name = ' '.join(row.contents)
    
    # Filter the digit and empty names
    if not(colunm_name.strip().isdigit()):
        colunm_name = colunm_name.strip()
        return colunm_name    


In [33]:
import pandas as pd
import requests

# URL da versão estática da página da Wikipedia
static_url = "https://en.wikipedia.org/w/index.php?title=List_of_Falcon_9_and_Falcon_Heavy_launches&oldid=1027686922"

# Cabeçalhos para simular um navegador
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
                  "AppleWebKit/537.36 (KHTML, like Gecko) "
                  "Chrome/91.0.4472.124 Safari/537.36"
}

# Faz a requisição HTTP
response = requests.get(static_url, headers=headers)

# Verifica se a requisição foi bem-sucedida
if response.status_code == 200:
    # Lê todas as tabelas da página HTML
    html_tables = pd.read_html(response.text)

    # Mostra quantas tabelas foram encontradas
    print(f"Número de tabelas encontradas: {len(html_tables)}")

    # Exibe a terceira tabela (índice 2), se existir
    if len(html_tables) > 2:
        first_launch_table = html_tables[2]
        print(first_launch_table.head())
    else:
        print("Menos de 3 tabelas encontradas na página.")
else:
    print(f"Erro ao acessar a página: {response.status_code}")

Número de tabelas encontradas: 25
   Flight No.                                 Date andtime (UTC)  \
0           1                                  4 June 2010,18:45   
1           1  First flight of Falcon 9 v1.0.[11] Used a boil...   
2           2                          8 December 2010,15:43[13]   
3           2  Maiden flight of Dragon capsule, consisting of...   
4           3                              22 May 2012,07:44[17]   

                                 Version,Booster [b]  \
0                               F9 v1.0[7]B0003.1[8]   
1  First flight of Falcon 9 v1.0.[11] Used a boil...   
2                               F9 v1.0[7]B0004.1[8]   
3  Maiden flight of Dragon capsule, consisting of...   
4                               F9 v1.0[7]B0005.1[8]   

                                         Launch site  \
0                                       CCAFS,SLC-40   
1  First flight of Falcon 9 v1.0.[11] Used a boil...   
2                                       CCAFS,SLC-40

In [34]:
first_launch_table = html_tables[2]
print(first_launch_table)

    Flight No.                                 Date andtime (UTC)  \
0            1                                  4 June 2010,18:45   
1            1  First flight of Falcon 9 v1.0.[11] Used a boil...   
2            2                          8 December 2010,15:43[13]   
3            2  Maiden flight of Dragon capsule, consisting of...   
4            3                              22 May 2012,07:44[17]   
5            3  Dragon spacecraft demonstrated a series of tes...   
6            4                           8 October 2012,00:35[21]   
7            4                           8 October 2012,00:35[21]   
8            4  CRS-1 was successful, but the secondary payloa...   
9            5                                 1 March 2013,15:10   
10           5  Last launch of the original Falcon 9 v1.0 laun...   
11           6                        29 September 2013,16:00[30]   
12           6  First commercial mission with a private custom...   
13           7                    

In [35]:
print(first_launch_table.columns)

Index(['Flight No.', 'Date andtime (UTC)', 'Version,Booster [b]',
       'Launch site', 'Payload[c]', 'Payload mass', 'Orbit', 'Customer',
       'Launchoutcome', 'Boosterlanding'],
      dtype='object')


In [37]:
column_names = first_launch_table.columns.tolist()
launch_dict = dict.fromkeys(column_names)
print(launch_dict)

{'Flight No.': None, 'Date andtime (UTC)': None, 'Version,Booster [b]': None, 'Launch site': None, 'Payload[c]': None, 'Payload mass': None, 'Orbit': None, 'Customer': None, 'Launchoutcome': None, 'Boosterlanding': None}


In [38]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# Funções auxiliares
def date_time(cell):
    text = cell.text.strip()
    parts = text.split()
    date = " ".join(parts[:3]) if len(parts) >= 3 else None
    time = parts[3] if len(parts) >= 4 else None
    return [date, time]

def booster_version(cell):
    return cell.text.strip() if cell else None

def get_mass(cell):
    try:
        return cell.text.strip().split()[0]
    except:
        return None

def landing_status(cell):
    return cell.text.strip() if cell else None

# URL da versão estática da página da Wikipedia
url = "https://en.wikipedia.org/w/index.php?title=List_of_Falcon_9_and_Falcon_Heavy_launches&oldid=1027686922"

headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
                  "AppleWebKit/537.36 (KHTML, like Gecko) "
                  "Chrome/91.0.4472.124 Safari/537.36"
}

# Requisição e parsing
response = requests.get(url, headers=headers)
soup = BeautifulSoup(response.text, "html.parser")

# Nomes das colunas e dicionário de dados
column_names = ['Flight No.', 'Date', 'Time', 'Version Booster', 'Launch Site',
                'Payload', 'Payload mass', 'Orbit', 'Customer', 'Launch outcome', 'Booster landing']
launch_dict = {key: [] for key in column_names}

# Extração das tabelas
extracted_row = 0
for table_number, table in enumerate(soup.find_all('table', {'class': 'wikitable plainrowheaders collapsible'})):
    for rows in table.find_all("tr"):
        if rows.th and rows.th.string:
            flight_number = rows.th.string.strip()
            flag = flight_number.isdigit()
        else:
            flag = False

        row = rows.find_all('td')

        if flag and len(row) >= 9:
            extracted_row += 1
            launch_dict['Flight No.'].append(flight_number)

            datetimelist = date_time(row[0])
            launch_dict['Date'].append(datetimelist[0])
            launch_dict['Time'].append(datetimelist[1])

            bv = booster_version(row[1]) or (row[1].a.string if row[1].a else None)
            launch_dict['Version Booster'].append(bv)

            launch_site = row[2].a.string if row[2].a else None
            launch_dict['Launch Site'].append(launch_site)

            payload = row[3].a.string if row[3].a else None
            launch_dict['Payload'].append(payload)

            payload_mass = get_mass(row[4])
            launch_dict['Payload mass'].append(payload_mass)

            orbit = row[5].a.string if row[5].a else None
            launch_dict['Orbit'].append(orbit)

            customer = row[6].a.string if row[6].a else None
            launch_dict['Customer'].append(customer)

            launch_outcome = list(row[7].strings)[0] if row[7].strings else None
            launch_dict['Launch outcome'].append(launch_outcome)

            booster_landing = landing_status(row[8])
            launch_dict['Booster landing'].append(booster_landing)

# Converter para DataFrame e exibir
df = pd.DataFrame(launch_dict)
print(f"Total de lançamentos extraídos: {extracted_row}")
print(df.head())

Total de lançamentos extraídos: 121
  Flight No.                       Date  Time       Version Booster  \
0          1          4 June 2010,18:45  None  F9 v1.0[7]B0003.1[8]   
1          2  8 December 2010,15:43[13]  None  F9 v1.0[7]B0004.1[8]   
2          3      22 May 2012,07:44[17]  None  F9 v1.0[7]B0005.1[8]   
3          4   8 October 2012,00:35[21]  None  F9 v1.0[7]B0006.1[8]   
4          5         1 March 2013,15:10  None  F9 v1.0[7]B0007.1[8]   

  Launch Site                               Payload Payload mass Orbit  \
0       CCAFS  Dragon Spacecraft Qualification Unit         None   LEO   
1       CCAFS                                Dragon         None   LEO   
2       CCAFS                                Dragon          525   LEO   
3       CCAFS                          SpaceX CRS-1        4,700   LEO   
4       CCAFS                          SpaceX CRS-2        4,877   LEO   

  Customer Launch outcome            Booster landing  
0   SpaceX      Success\n  Failure[9]

In [39]:
df= pd.DataFrame({ key:pd.Series(value) for key, value in launch_dict.items() })

In [44]:
df.to_csv('spacex_web_scraped.csv', index=False)